# 5장 - 클래스와 인터페이스


- 객체지향 언어로서 파이썬은 상속(inheritance), 다형성(polymorphism), 캡슐화(encapsulation)등과 같은 모든 기능을 제공한다.

### BETTER WAY 37 - 내장 타입을 여러 단계로 내포시키기보다는 클래스를 합성하라

- 파이썬 내장 딕셔너리 타입을 사용하면 객체의 생명 주기 동안 동적인 내부상태를 잘 유지할 수 있다.

In [1]:
class SimpleGradebook:
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = []
    
    def report_grade(self, name, score):
        self._grades[name].append(score)
        
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)
    

book = SimpleGradebook()
book.add_student('뉴턴')
book.report_grade('뉴턴', 90)
book.report_grade('뉴턴', 95)
book.report_grade('뉴턴', 85)
print(book.average_grade('뉴턴'))

90.0


In [4]:
'''
딕셔너리와 관련 내장 타입은 사용하기 너무 쉬우므로 과하게 확장하면서 깨지기 쉬운 코드를 작성할 위험성이 있다.
요구사항이 바뀌어 SimpleGradebook 클래스를 전체 성적이 아니라, 과목별 성적을 리스트로 저장하고 싶다고 하자.
'''
from collections import defaultdict

class BySubjectGradebook:
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = defaultdict(list)
    
    def report_grade(self, name, subject, score):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append(score)
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return total / count
    
book = BySubjectGradebook()
book.add_student('뉴턴')
book.report_grade('뉴턴', '수학', 75)
book.report_grade('뉴턴', '수학', 65)
book.report_grade('뉴턴', '체육', 90)
book.report_grade('뉴턴', '체육', 95)
print(book.average_grade('뉴턴'))

81.25


In [6]:
'''
요구사항이 또 바뀌어, 이번엔 각 점수의 가중치를 함께 저장해서 중간고사와 기말고사가 더 큰 영향을 미치게 하고 싶다.
'''
class WeightedGradebook:
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = defaultdict(list)
    
    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append((score, weight))
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        
        score_sum, score_count = 0, 0
        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0
            
            for score, weight in scores:
                subject_avg += score * weight
                total_weight += weight
            
            score_sum += subject_avg / total_weight
            score_count += 1
        
        return score_sum / score_count
    
book = WeightedGradebook()
book.add_student('뉴턴')
book.report_grade('뉴턴', '수학', 75, 0.05)
book.report_grade('뉴턴', '수학', 65, 0.15)
book.report_grade('뉴턴', '수학', 70, 0.80)
book.report_grade('뉴턴', '체육', 100, 0.40)
book.report_grade('뉴턴', '체육', 85, 0.60)
print(book.average_grade('뉴턴'))

80.25


- 하지만, 클래스도 쓰기 어려워졌다. 위치로 인자를 지정하면, 어떤 값이 어떤 뜻을 가지는지 이해하기 어렵다.
- 이와 같은 복잡도가 눈에 들어오면 더 이상 딕셔너리, 튜플, 집합, 리스트 등의 내장 타입을 사용하지 말고, 클래스 계층 구조를 사용해야 한다.

- <font color='blue'>클래스를 활용해 리팩터링하기</font>

- collection 내장 모듈에 있는 namedtuple 타입이 이런 경우에 딱 들어맞는다.
- namedtuple 을 사용하면 작은 불변 데이터 클래스를 쉽게 정의할 수 있다.

In [9]:
from collections import namedtuple

# 
Grade = namedtuple('Grade', ('score', 'weight'))

# 일련의 점수를 포함한 단일 과목을 표현하는 클래스 작성
class Subject:
    def __init__(self):
        self._grades = []
        
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
        
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight
    
# 한 학생이 수강하는 과목들을 표현하는 클래스
class Student:
    def __init__(self):
        self._subjects = defaultdict(Subject)
    
    def get_subject(self, name):
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count
    
# 마지막으로 모든 학생을 저장하는 컨테이너
class Gradebook:
    def __init__(self):
        self._students = defaultdict(Student)
        
    def get_student(self, name):
        return self._students[name]

# 다소 길지만 훨씬 더 보기 좋을 것이다.
#
book = Gradebook()
albert = book.get_student('아인슈타인')
math = albert.get_subject('수학')
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)

gym = albert.get_subject('체육')
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)

print(albert.average_grade())

80.25


### BETTER WAY 38 - 간단한 인터페이스의 경우 클래스 대신 함수를 받아라

- 파이썬 내장 API 중 상당수는 함수를 전달해서 동작을 원하는 대로 바꿀 수 있게 해준다.
- API 가 실행되는 과정에서 여러분이 전달한 함수를 실행하는 경우, 이런 함수를 <font color='blue'>훅(hook)</font> 이라고 부른다.

In [2]:
# 예) 리스트 타입의 sort 메서드에 함수 지정
names = ['소크라테스', '아르키메데스', '플라톤', '아리스토텔레스']
names.sort()
print(names)
names.sort(key=len)
print(names)

['소크라테스', '아르키메데스', '아리스토텔레스', '플라톤']
['플라톤', '소크라테스', '아르키메데스', '아리스토텔레스']


- 파이썬은 함수를 <font color='blue'>일급 시민 객체</font>로 취급하기 때문에 함수를 훅으로 사용할 수 있다.
- 함수나 메서드가 일급 시민 객체란 말은, 파이썬 언어에서 사용할 수 있는 다른 일반적인 값과 마찬가지로 함수나 메서드를 다른 함수에 넘기거나 변수 등으로 참조할 수 있다는 의미다.

### BETTER WAY 39 - 객체를 제너릭하게 구성하려면 @classmethod 를 통한 다형성을 활용하라

- 파이썬은 객체뿐 아니라 클래스도 다형성을 지원한다.
- 다형성을 사용하면 계층을 이루는 여러 클래스가 자신에게 맞는 유일한 메서드 버전을 구현할 수 있다.

In [4]:
# 공통 클래스
class InputData:
    def read(self):
        raise NotImplementedError

# InputData 의 구체적인 하위 클래스
class PathInputData(InputData):
    def __init__(self, path):
        super().__init__()
        self.path = path
        
    def read(self):
        with open(self.path) as f:
            return f.read()
        
# 이런식으로 InputData 의 하위 클래스로 무궁무진하게 만들 수 있다.
# 각 하위 클래스는 처리할 데이터를 돌려주는 공통 read 인터페이스를 구현해야 한다.

In [5]:
# 입력데이터를 소비하는 공통 클래스
class Worker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None
        
    def map(self):
        raise NotImplementedError
        
    def reduce(self, other):
        raise NotImplementedError
        
# Worker 의 구체적인 하위 클래스
class LineCountWorker(Worker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count("\n")
        
    def reduce(self, other):
        self.result += other.result

In [7]:
# 이 구현은 아주 잘 동작할 것처럼 보이지만, 모든 요소를 구현하는 과정에서 큰 난관에 부딪혔다.
# 대체 각 부분을 어떻게 연결해야 할까?
# 이해하기 쉬운 인터페이스와 추상화를 제공하는 멋진 클래스를 여럿 만들었지만, 객체를 생성해 활용해야만 이 모든 클래스가 쓸 모 있게 된다.

# 각 객체를 만들고 맵리듀스를 조화롭게 실행하는 책임은 누가 져야 할까?

'''
가장 간단한 접근 방법은 도우미 함수를 활용해 객체를 직접 만들고 연결하는 것
'''

import os

# 디렉터리 목록을 얻어서 그 안에 들어 있는 파일마다 PathInputData 인스턴스를 만드는 함수
def generate_inputs(data_dir):
    for name in os.listdir(data_dir):
        yield PathInputData(os.path.join(data_dir, name))
        
#
def create_workers(input_list):
    workers = []
    for input_data in input_list:
        workers.append(LineCountWorker(input_data))
    return workers

#
from threading import Thread

def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
        
    first, *rest = workers
    for worker in rest:
        first.reduce(worker)
    return first.result

# 마지막으로 지금까지 만든 모든 조각을 한 함수 안에 합쳐서 각 단계를 실행
def mapreduce(data_dir):
    inputs = generate_inputs(data_dir)
    workers = create_workers(inputs)
    return execute(workers)

In [14]:
####################
# 이 함수를 실행해보면 아주 훌륭하게 동작하지만, 문제가 있다.
# 바로 앞에서 정의한 mapreduce 함수의 가장 큰 문제점은 함수가 전혀 제너릭(generic)하지 않다는 것이다.

# 다른 InputData 나 Woker 하위 클래스를 사용하고 싶으면 그에 맞게 재작성해야 한다는 것이다.

# 이 문제의 핵심은, 객체를 구성할 수 있는 제너릭한 방법이 필요하다는 점이다.

# 다른 언어에서는 다형성을 활용해 이 문제를 해결할 수 있다.
# InputData 의 모든 하위 클래스는 맵리듀스를 처리하는 도우미 메서들이 제너릭하게 사용할 수 있는 특별한 생성자를 제공한다.

# 반면, 파이썬에서는 생성자 메서드가 __init__ 밖에 없다는 점이 문제다.
# 이 문제를 해결하는 가장 좋은 방법은 클래스 메서드 다형성을 사용하는 것이다.

# InputData 에 제너릭 @classmethod 를 적용한 모습
class GenericInputData:
    def read(self):
        raise NotImplementedError
        
    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError

class PathInputData(GenericInputData):
    def __init__(self, path):
        super().__init__()
        self.path = path
        
    def read(self):
        with open(self.path) as f:
            return f.read()
        
    @classmethod
    def generate_inputs(cls, config):
        data_dir = config['data_dir']
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))
            
# Worker 에 제너릭 @classmethod 를 적용한 모습
class GenericWorker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None
        
    def map(self):
        raise NotImplementedError
        
    def reduce(self, other):
        raise NotImplementedError

    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))
        return workers

class LineCountWorker(GenericWorker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count("\n")
        
    def reduce(self, other):
        self.result += other.result

from threading import Thread

def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
        
    first, *rest = workers
    for worker in rest:
        first.reduce(worker)
    return first.result

# 마지막으로 mapreduct 함수가 create_workers 를 호출하게 변경해서 mapreduce를 완전한 제너릭 함수로 만들 수 있다.
def mapreduce(worker_class, input_class, config):
    workers = worker_class.create_workers(input_class, config)
    return execute(workers)

# 제너릭하게 작동해야 하므로 mapreduce 함수에 더 많은 파라미터를 넘겨야 한다는 것 뿐이다.
tmpdir = 'test_inputs'
config = {'data_dir': tmpdir}
result = mapreduce(LineCountWorker, PathInputData, config)

### BETTER WAY 40 - super로 부모 클래스를 초기화하라

- 자식 클래스에서 부모 클래스를 초기화하는 오래된 방법은 바로 자식 인스턴스에서 부모 클래스의 \__init__ 메서드를 직접 호출하는 것이다.
- 이 접근 방법은 기본적인 클래스 계층의 경우에는 잘 동작하지만, 다른 경우에는 잘못 될 수도 있다.

- 어떤 클래스가 다중 상속에 의해 영향을 받는 경우, 상위 클래스의 \__init__ 메서드를 직접 호출하면 프로그램이 예측할 수 없는 방식으로 작동할 수 있다.
- 다중 상속을 사용하는 경우 생기는 문제 중 하나는 모든 하위클래스에서 \__init__ 호출의 순서가 정해져 있지 않다는 것이다. 

In [15]:
class MyBaseClass:
    def __init__(self, value):
        self.value = value
        
class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)

In [18]:
class TimesTwo:
    def __init__(self):
        self.value *= 2
        
class PlusFive:
    def __init__(self):
        self.value += 5

class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)
        
foo = OneWay(5)
print(foo.value) # (5 * 2) + 5

15


In [20]:
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

foo = AnotherWay(5)
print(foo.value) # (5 * 2) + 5

15


In [21]:
# OneWay 와 AnotherWay 에 대해서는 같은 결과를 보인다.
# 이는, 클래스 정의에서 부모 클래스를 나열한 순서와 부모 클래스의 생성자를 호출하는 순서가 일치하지 않는다는 것을 의미한다.

# 다이아몬드 상속으로 인해 다른 문제가 생길 수 도 있다.
# 다이아몬드 상속이란, 어떤 클래스가 두 가지 서로 다른 클래스를 상속하는데, 두 상의 클래스의 상속 계층을 거슬러 올라가면 같은 조상 클래스가 존재하는 경우를 뜻한다.

class TimesSeven(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 7
        
class PlusNine(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 9
        
class ThisWay(TimesSeven, PlusNine):
    def __init__(self, value):
        TimesSeven.__init__(self, value)
        PlusNine.__init__(self, value)
        
foo = ThisWay(5)
print(foo.value)

14


In [33]:
# 기대한 값은 5*7 + 9 = 44 이지만, 14가 나온다.
# 이는 두 번째 부모 클래스의 생성자 PlusNine.__init__ 을 호출하면서 self.value 가 5 로 돌아갔기 때문이다.


# 이러한 문제를 해결하기 위해 파이썬에서는 super 라는 내장 함수와 표준 메서드 결정 순서 (Method Resolution Order, MRO) 가 있다.
# super 를 사용하면 다이아몬드 계층의 공통 상위 클래스를 단 한 번만 호출하도록 보장한다.
# MRO 는 상위 클래스를 초기화하는 순서를 정의한다. 이 때 C3 선형화(linearization)라는 알고리즘을 사용한다.

class TimesSevenCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value *= 7
        
class PlusNineCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value += 9
        
class ThisWay(TimesSevenCorrect, PlusNineCorrect):
    def __init__(self, value):
        super().__init__(value)
        
foo = ThisWay(5)
print(foo.value)

98


In [26]:
# 결과는 7 * (5 + 9) = 98 이므로 98 이 나왔다.

# 응?
# (7 * 5) + 9 = 44 가 나와야 하는거 아닌가???

# 대답은 "아니오" 다.
# 호출 순서는 이 클래스에 대한 MRO 정의를 따른다. MRO 순서는 mro 라는 클래스 메서드를 통해 살펴볼 수 있다.

print(ThisWay.mro())

[<class '__main__.ThisWay'>, <class '__main__.TimesSevenCorrect'>, <class '__main__.PlusNineCorrect'>, <class '__main__.MyBaseClass'>, <class 'object'>]


In [35]:
# 즉,
# ThisWay -> TimesSevenCorrect -> PlusNineCorrect -> MyBaseClass 를 호출한다.
# 각 초기화 메서드는 호출된 순서의 역순으로 작업을 수행하므로, 
# MyBaseClass 에 의해서 value 가 5 로 설정되고
# PlusNineCorrect 에 의해서 value 에 +9 가 되어 14로 만들고
# TimesSevenCorrect 에 의해서 value 에 *7 이 되어 98이 만들어 지는 것이다.

# 만약, ThisWay2 클래스 정의 시 상속 순서를 뒤집어서 작성하면, 44 가 나온다.


class ThisWay2(PlusNineCorrect, TimesSevenCorrect):
    def __init__(self, value):
        super().__init__(value)

foo2 = ThisWay2(5)
print('ThisWay:', foo2.value)

ThisWay: 44


### BETTER WAY 41 - 기능을 합성할 때는 믹스인 클래스를 사용하라

- 파이썬은 다중 상속을 처리할 수 있게 지원하는 객체지향 언어이지만, 다중 상속은 피하는 편이 좋다.
- 다중 상속이 제공하는 편의와 캡슐화가 필요하지만, 다중 상속으로 인해 발생할 수 있는 골치 아픈 경우는 피하고 싶다면
- 믹스인(mix-in)을 사용할지 고려행 보라.

- <font color='blue'>믹스인은 자식 클래스가 사용할 메서드 몇 개만 정의하는 클래스다.</font>
- 믹스인 클래스에는 자체 attribute 정의가 없으므로 믹스인 클래스의 \__init__ 메서드를 호출할 필요도 없다.

In [7]:
# 예)
# 메모리 내에 들어 있는 파이썬 객체를 직렬화에 사용할 수 있도록 딕셔너리로 바꾸고 싶다.
# 이런 기능을 제너릭하게 작성해 여러 클래스에서 활용하면 어떨까?

class ToDictMixin:
    def to_dict(self):
        return self._traverse_dict(self.__dict__)
    
    # 이 _traverse_dict 메서드를 hasattr를 통한 동적인 attribute 접근과 isinstance를 사용한 타입 검사, __dict__를 통한 인스턴스 딕셔너리 접근을 활용해 간단하게 구현할 수 있다.

    def _traverse_dict(self, instance_dict):
        output = {}
        for key, value in instance_dict.items():
            output[key] = self._traverse(key, value)
        return output
            
    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        elif hasattr(value, '__dict__'):
            return self._traverse_dict(value.__dict__)
        else:
            return value
        
# 이 믹스인을 사용해 이진 트리를 딕셔너리 표현으로 변경하는 예제코드다
class BinaryTree(ToDictMixin):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        
# 연관된 여러 파이썬 객체들을 한 딕셔너리로 변환하는 것도 쉽게 할 수 있다.
#tree = BinaryTree(10)
#print(tree.to_dict())

tree = BinaryTree(10, 
                  left=BinaryTree(7, right=BinaryTree(9)),
                  right=BinaryTree(13, left=BinaryTree(11)))
print(tree.to_dict())

{'value': 10, 'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None}}, 'right': {'value': 13, 'left': {'value': 11, 'left': None, 'right': None}, 'right': None}}


In [ ]:
class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, left=None, right=None, parent=None):
        super().__init__(value, left=left, right=right)
        self.parent = parent